# Preparação do banco de dados CAPES 


## Banco de Dados com informações sobre bolsas de mestrado, doutorado no Brasil de 1995 até 2018.
A fonte de dados é GEOCAPES https://geocapes.capes.gov.br/geocapes/

Porém os dados utilizados nessa preparação é do Kaggle https://www.kaggle.com/datasets/voltdata/capes-bolsas/data

## Importação das bibliotecas e da base de dados

In [74]:
#!pip install unidecode

In [75]:
import pandas as pd
import numpy as np
from unidecode import unidecode

In [76]:
capes = pd.read_csv(r"C:\Users\joaov\OneDrive\Área de Trabalho\capes_bolsas.csv")
capes.head()
##exportando em csv para facilitar a analise no sql 
##capes_bolsas.to_csv('capes_bolsas.csv', index=False)

,Unnamed: 0,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,Codigo IES,Coordenador Geral IsF,Coordenador Pedagógico IsF,DOUTORADO PLENO,Iniciação Científica,JOVENS TALENTOS - A,JOVENS TALENTOS - B,MESTRADO,MESTRADO PROFISSIONAL,PESQUISADOR VISITANTE ESPECIAL,PROF VISITANTE NACIONAL SÊNIOR,PROFESSOR VISITANTE EXTERIOR PLENO,PROFESSOR VISITANTE EXTERIOR SENIOR,Professor IsF,PÓS-DOUTORADO,Total Linha
0,1,2018,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,11001011.0,0.0,0.0,1,0.0,0.0,0.0,38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39
1,2,2018,AC,RIO BRANCO,NORTE,NaN,ISF,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,11001011.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,3,2018,AC,RIO BRANCO,NORTE,NaN,PROCAD,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,11001011.0,0.0,0.0,0,6.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
3,4,2018,AC,RIO BRANCO,NORTE,NaN,PVNS,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,11001011.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
4,5,2018,AC,RIO BRANCO,NORTE,11001011001P8,DS,UFAC,Federal,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,INDEFINIDA,ECOLOGIA,CIÊNCIAS BIOLÓGICAS,11001011.0,0.0,0.0,0,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8


## Verificando as informações da base de dados

Das informações númericas, somente Mestrado e Doutorado não apresentam valores faltantes, enquanto todas as outras apresentam uma alta ocorrência de informações faltantes.

In [77]:
# Quais informações apresentam informações nulas
capes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136114 entries, 0 to 136113
Data columns (total 29 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Unnamed: 0                           136114 non-null  int64  
 1   Ano                                  136114 non-null  int64  
 2   UF                                   136114 non-null  object 
 3   Município                            136114 non-null  object 
 4   Região                               136114 non-null  object 
 5   Código Programa                      130415 non-null  object 
 6   Programa Fomento                     136114 non-null  object 
 7   IES                                  136113 non-null  object 
 8   Status Jurídico                      136114 non-null  object 
 9   Programa                             130415 non-null  object 
 10  Área Avaliação                       136114 non-null  object 
 11  Área Conhecim

In [78]:
capes.columns

Index(['Unnamed: 0', 'Ano', 'UF', 'Município', 'Região', 'Código Programa',
       'Programa Fomento', 'IES', 'Status Jurídico', 'Programa',
       'Área Avaliação', 'Área Conhecimento', 'Grande Área', 'Codigo IES',
       'Coordenador Geral IsF', 'Coordenador Pedagógico IsF',
       'DOUTORADO PLENO', 'Iniciação Científica', 'JOVENS TALENTOS - A',
       'JOVENS TALENTOS - B', 'MESTRADO', 'MESTRADO PROFISSIONAL',
       'PESQUISADOR VISITANTE ESPECIAL', 'PROF VISITANTE NACIONAL SÊNIOR',
       'PROFESSOR VISITANTE EXTERIOR PLENO',
       'PROFESSOR VISITANTE EXTERIOR SENIOR', 'Professor IsF', 'PÓS-DOUTORADO',
       'Total Linha'],
      dtype='object')

In [79]:
## Relação de não faltantes com o ano
qtd_nulos = pd.DataFrame()
for column in capes.columns[13:]:   ## 13 pois a var de corte para as númericas
    null_counts = capes.groupby('Ano')[column].apply(lambda x: x.isnull().sum())
    non_null_counts = capes.groupby('Ano')[column].apply(lambda x: x.notnull().sum())
    qtd_nulos[f'nulos_{column}'] = null_counts
    qtd_nulos[f'não_nulos_{column}'] = non_null_counts

pd.set_option('display.max_columns', None)
qtd_nulos

,nulos_Codigo IES,não_nulos_Codigo IES,nulos_Coordenador Geral IsF,não_nulos_Coordenador Geral IsF,nulos_Coordenador Pedagógico IsF,não_nulos_Coordenador Pedagógico IsF,nulos_DOUTORADO PLENO,não_nulos_DOUTORADO PLENO,nulos_Iniciação Científica,não_nulos_Iniciação Científica,nulos_JOVENS TALENTOS - A,não_nulos_JOVENS TALENTOS - A,nulos_JOVENS TALENTOS - B,não_nulos_JOVENS TALENTOS - B,nulos_MESTRADO,não_nulos_MESTRADO,nulos_MESTRADO PROFISSIONAL,não_nulos_MESTRADO PROFISSIONAL,nulos_PESQUISADOR VISITANTE ESPECIAL,não_nulos_PESQUISADOR VISITANTE ESPECIAL,nulos_PROF VISITANTE NACIONAL SÊNIOR,não_nulos_PROF VISITANTE NACIONAL SÊNIOR,nulos_PROFESSOR VISITANTE EXTERIOR PLENO,não_nulos_PROFESSOR VISITANTE EXTERIOR PLENO,nulos_PROFESSOR VISITANTE EXTERIOR SENIOR,não_nulos_PROFESSOR VISITANTE EXTERIOR SENIOR,nulos_Professor IsF,não_nulos_Professor IsF,nulos_PÓS-DOUTORADO,não_nulos_PÓS-DOUTORADO,nulos_Total Linha,não_nulos_Total Linha
Ano,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995,1723,0,1723,0,1723,0,0,1723,1723,0,1723,0,1723,0,0,1723,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,0,1723
1996,1810,0,1810,0,1810,0,0,1810,1810,0,1810,0,1810,0,0,1810,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,0,1810
1997,1926,0,1926,0,1926,0,0,1926,1926,0,1926,0,1926,0,0,1926,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,0,1926
1998,1977,0,1977,0,1977,0,0,1977,1977,0,1977,0,1977,0,0,1977,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,0,1977
1999,1965,0,1965,0,1965,0,0,1965,1965,0,1965,0,1965,0,0,1965,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,0,1965
2000,2175,0,2175,0,2175,0,0,2175,2175,0,2175,0,2175,0,0,2175,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,0,2175,0,2175
2001,2338,0,2338,0,2338,0,0,2338,2338,0,2338,0,2338,0,0,2338,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,0,2338,0,2338
2002,2606,0,2606,0,2606,0,0,2606,2606,0,2606,0,2606,0,0,2606,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,0,2606,0,2606
2003,4390,0,4390,0,4390,0,0,4390,4390,0,4390,0,4390,0,0,4390,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,0,4390,0,4390


Fazendo a análise da quantidade de nulos por ano para cada variável percebemos que temos algumas que passaram a ser contados somente em 2018, Como o código do IES (Instituição de Ensino Superior), professores visitantes do exterior. E algumas variáveis são contadas somente apartir de um momento, como o Coordenador Geral do ISF e informações de Iniciação Cientifica tem somente dados de 2013 para frente, Coordenador Pedagógico IsF de 2014, entre outras informações que só aparecem depois de um certo momento, logo se essas informações forem objeto de estudo é interessante separar para somente aqueles anos em questão. No meu caso, irei dropar essas informações, já que irei focar somente em mestrado e doutorado 

In [80]:
## Mas algo que pode ser feito NESSE DATAFRAME, é transformar em zero porém não é um objeto de análise olhar para esses zeros.
colunas_0 = ['Codigo IES',
       'Coordenador Geral IsF', 'Coordenador Pedagógico IsF','Iniciação Científica', 'JOVENS TALENTOS - A',
       'JOVENS TALENTOS - B', 'MESTRADO PROFISSIONAL',
       'PESQUISADOR VISITANTE ESPECIAL', 'PROF VISITANTE NACIONAL SÊNIOR',
       'PROFESSOR VISITANTE EXTERIOR PLENO',
       'PROFESSOR VISITANTE EXTERIOR SENIOR', 'Professor IsF', 'PÓS-DOUTORADO']
type(colunas_0)

list

In [81]:
## Transformando os NA em Zero, mostrando como pode ser feito
capes[colunas_0] = capes[colunas_0].fillna(0)
capes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136114 entries, 0 to 136113
Data columns (total 29 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Unnamed: 0                           136114 non-null  int64  
 1   Ano                                  136114 non-null  int64  
 2   UF                                   136114 non-null  object 
 3   Município                            136114 non-null  object 
 4   Região                               136114 non-null  object 
 5   Código Programa                      130415 non-null  object 
 6   Programa Fomento                     136114 non-null  object 
 7   IES                                  136113 non-null  object 
 8   Status Jurídico                      136114 non-null  object 
 9   Programa                             130415 non-null  object 
 10  Área Avaliação                       136114 non-null  object 
 11  Área Conhecim

In [82]:
## Porém para esse dataframe essas informações não são interessantes de manter
capes_bolsas = capes.drop(columns=['Unnamed: 0','Codigo IES',
       'Coordenador Geral IsF', 'Coordenador Pedagógico IsF','Iniciação Científica', 'JOVENS TALENTOS - A',
       'JOVENS TALENTOS - B', 'MESTRADO PROFISSIONAL',
       'PESQUISADOR VISITANTE ESPECIAL', 'PROF VISITANTE NACIONAL SÊNIOR',
       'PROFESSOR VISITANTE EXTERIOR PLENO',
       'PROFESSOR VISITANTE EXTERIOR SENIOR', 'Professor IsF', 'PÓS-DOUTORADO', 'Total Linha'])
capes_bolsas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136114 entries, 0 to 136113
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Ano                136114 non-null  int64 
 1   UF                 136114 non-null  object
 2   Município          136114 non-null  object
 3   Região             136114 non-null  object
 4   Código Programa    130415 non-null  object
 5   Programa Fomento   136114 non-null  object
 6   IES                136113 non-null  object
 7   Status Jurídico    136114 non-null  object
 8   Programa           130415 non-null  object
 9   Área Avaliação     136114 non-null  object
 10  Área Conhecimento  136114 non-null  object
 11  Grande Área        136114 non-null  object
 12  DOUTORADO PLENO    136114 non-null  int64 
 13  MESTRADO           136114 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 14.5+ MB


Agora temos informações faltantes, em Código do Programa e Programa na mesma quantidade, e tem um faltante em IES, para corrigir o IES podemos encontrar ele no dataframe e comparar ver de qual múnicipio ele é, e quais IES tem aquele municipio.

In [83]:
capes_bolsas[capes_bolsas['IES'].isnull()]
## Somente em Santarém no Pará, logo podemos procurar para ver todas as IES de Santarém

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,DOUTORADO PLENO,MESTRADO
3446,2018,PA,SANTARÉM,NORTE,33003017001P2,PROCAD,NaN,Federal,EDUCAÇÃO,INDEFINIDA,EDUCAÇÃO,CIÊNCIAS HUMANAS,0,0


In [84]:
capes_bolsas.loc[capes_bolsas['Município'] == 'SANTARÉM', 'IES'].unique()
## Como só temos UFOPA e UFOPA Santarém, sao duas informações vinda da mesma faculdade, UFOPA de Santarem
## Logo irei substituir por ufopa somente e depois substituir todas ufopa santarem por ufopa, já que eu tenho o municipio, mais para frente posso utilizar
## essa var para distinguir entre as ufopas 

array(['UFOPA', nan, 'UFOPA Santarém'], dtype=object)

Não irei corrigir os erros de programa e Código Programa por agora, irei deixar para tratar da maneira que eu for analisar no momento

In [85]:
capes_bolsas['IES'].replace({np.nan: 'UFOPA', 'UFOPA Santarém': 'UFOPA'}, inplace=True)

In [86]:
capes_bolsas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136114 entries, 0 to 136113
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Ano                136114 non-null  int64 
 1   UF                 136114 non-null  object
 2   Município          136114 non-null  object
 3   Região             136114 non-null  object
 4   Código Programa    130415 non-null  object
 5   Programa Fomento   136114 non-null  object
 6   IES                136114 non-null  object
 7   Status Jurídico    136114 non-null  object
 8   Programa           130415 non-null  object
 9   Área Avaliação     136114 non-null  object
 10  Área Conhecimento  136114 non-null  object
 11  Grande Área        136114 non-null  object
 12  DOUTORADO PLENO    136114 non-null  int64 
 13  MESTRADO           136114 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 14.5+ MB


In [87]:
capes_bolsas.head()

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,DOUTORADO PLENO,MESTRADO
0,2018,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,1,38
1,2018,AC,RIO BRANCO,NORTE,NaN,ISF,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,0,0
2,2018,AC,RIO BRANCO,NORTE,NaN,PROCAD,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,0,0
3,2018,AC,RIO BRANCO,NORTE,NaN,PVNS,UFAC,Federal,NaN,INDEFINIDA,Grande Area Não Informada,Grande Area Não Informada,0,0
4,2018,AC,RIO BRANCO,NORTE,11001011001P8,DS,UFAC,Federal,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,INDEFINIDA,ECOLOGIA,CIÊNCIAS BIOLÓGICAS,0,8


In [88]:
# Para transformar os caracteres especiais em normais e transformando tudo em upper para não ocorrer em erros de alguma linha ta e outra não 
capes_bolsas = capes_bolsas.applymap(lambda x: unidecode(str(x)) if isinstance(x, str) else x)
capes_bolsas = capes_bolsas.applymap(lambda x: str(x).upper() if isinstance(x, str) else x)

In [89]:
capes_bolsas.head()

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,DOUTORADO PLENO,MESTRADO
0,2018,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,1,38
1,2018,AC,RIO BRANCO,NORTE,NaN,ISF,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,0,0
2,2018,AC,RIO BRANCO,NORTE,NaN,PROCAD,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,0,0
3,2018,AC,RIO BRANCO,NORTE,NaN,PVNS,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,0,0
4,2018,AC,RIO BRANCO,NORTE,11001011001P8,DS,UFAC,FEDERAL,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,INDEFINIDA,ECOLOGIA,CIENCIAS BIOLOGICAS,0,8


In [90]:
# Renomeando as colunas
capes_bolsas.rename(columns={'Ano': 'ANO','Município': 'MUN', 'Região': 'REGIAO', 'Código Programa': 'COD_PROG', 'Programa Fomento': 'PROG_FORMENTO'
                            , 'Status Jurídico': 'STATUS_JURIDICO', 'Programa': 'PROGRAMA', 'Área Avaliação': 'AREA_AVALIACAO'
                            , 'Área Conhecimento': 'AREA_CONHECIMENTO', 'Grande Área': 'GRANDE_AREA', 'DOUTORADO PLENO': 'DOUTORADO'}, inplace=True)

In [91]:
## Verificando se algum caracter diferente seguiu na tabela
def encontrar_caracteres_especiais(df):
    caracteres_especiais = []
    for coluna in df.columns:
        for valor in df[coluna]:
            if isinstance(valor, str) and not valor.isascii():
                caracteres_especiais.append((coluna, valor))
    return caracteres_especiais

caracteres_problema = encontrar_caracteres_especiais(capes_bolsas)
print(caracteres_problema)
## Deu vazio o unicode limpou todas os caracteres especiais

[]


In [92]:
## O Drop precisa de uma lista de linhas do dataframe, ou seja, capes_bolsas[] passa a lista de linhas desse dataframe para o drop
## e dentro dessa lista, temos as condições, faz parte dessa lista, quando tanto doutorado e mestrado são = 0, o .index passa o index para o drop
capes_bolsas.drop(capes_bolsas[(capes_bolsas['DOUTORADO'] == 0) & (capes_bolsas['MESTRADO'] == 0)].index, inplace=True)
capes_bolsas
## Outra forma de fazer isso é usando o query, que faz um filtro utilizando as condições passadas, no caso quando um ou outro é 0.
## capes_bolsas = capes_bolsas.query('DOUTORADO != 0 or MESTRADO != 0')

,ANO,UF,MUN,REGIAO,COD_PROG,PROG_FORMENTO,IES,STATUS_JURIDICO,PROGRAMA,AREA_AVALIACAO,AREA_CONHECIMENTO,GRANDE_AREA,DOUTORADO,MESTRADO
0,2018,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,1,38
4,2018,AC,RIO BRANCO,NORTE,11001011001P8,DS,UFAC,FEDERAL,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,INDEFINIDA,ECOLOGIA,CIENCIAS BIOLOGICAS,0,8
6,2018,AC,RIO BRANCO,NORTE,11001011003P0,DS,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,INDEFINIDA,LETRAS,"LINGUISTICA, LETRAS E ARTES",0,17
8,2018,AC,RIO BRANCO,NORTE,11001011004P7,DS,UFAC,FEDERAL,PRODUCAO VEGETAL,INDEFINIDA,AGRONOMIA,CIENCIAS AGRARIAS,18,9
9,2018,AC,RIO BRANCO,NORTE,11001011004P7,FAPAC,UFAC,FEDERAL,PRODUCAO VEGETAL,INDEFINIDA,AGRONOMIA,CIENCIAS AGRARIAS,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136109,1995,SP,SAO PAULO,SUDESTE,33019010004P1,DS,FCMSCSP-TI,PRIVADA,MEDICINA (CLINICA MEDICA),MEDICINA I,CLINICA MEDICA,CIENCIAS DA SAUDE,0,3
136110,1995,SP,SAO PAULO,SUDESTE,33019010005P8,DS,FCMSCSP-TI,PRIVADA,MEDICINA (TOCOGINECOLOGIA),MEDICINA III,GINECOLOGIA E OBSTETRICIA,CIENCIAS DA SAUDE,0,6
136111,1995,SP,SAO PAULO,SUDESTE,33019010006P4,DS,FCMSCSP-TI,PRIVADA,MEDICINA (PEDIATRIA),MEDICINA II,PEDIATRIA,CIENCIAS DA SAUDE,5,7
136112,1995,SP,SAO PAULO,SUDESTE,33019010007P0,DS,FCMSCSP-TI,PRIVADA,MEDICINA (OTORRINOLARINGOLOGIA),MEDICINA III,CIRURGIA OTORRINOLARINGOLOGIA,CIENCIAS DA SAUDE,0,5


In [93]:
# média dos mestrandos ao longo desses anos
#para entender o filtro, ocorre pois primeiro eu tenho que filtrar o df só onde mestrado é > 0 e depois selecionar a coluna mestrado desse filtro
# caso contrário ele tenta fazer a média de todo o df filtrado, contando as colunas em texto
capes_bolsas[capes_bolsas['MESTRADO'] > 0]['MESTRADO'].mean()


8.308164375048552

In [94]:
#média de mestrado por ano 
## importante passar o  método .unique() pois passaria ano por ano
soma_por_ano = []
for ano in capes_bolsas['ANO'].unique():
    soma_mestrado = capes_bolsas[(capes_bolsas['MESTRADO'] > 0) & (capes_bolsas['ANO'] == ano)]['MESTRADO'].sum()
    soma_doutorado = capes_bolsas[(capes_bolsas['DOUTORADO'] > 0) & (capes_bolsas['ANO'] == ano)]['DOUTORADO'].sum()
    soma_por_ano.append((ano, soma_mestrado,soma_doutorado))
soma_ano_df = pd.DataFrame(soma_por_ano, columns=['ANO', 'soma_MESTRADO','soma_DOUTORADO'])
soma_ano_df

,ANO,soma_MESTRADO,soma_DOUTORADO
0,2018,47478,44530
1,2017,47510,44316
2,2016,47830,43188
3,2015,49353,42779
4,2014,48113,39954
5,2013,45754,32111
6,2012,43591,27589
7,2011,41054,26108
8,2010,33357,21941
9,2009,27192,17873


In [95]:
## outra forma de fazer é pelo grupby por ano
capes_bolsas[capes_bolsas['MESTRADO'] > 0].groupby('ANO')['MESTRADO'].mean().round(2)

ANO
1995     7.83
1996     7.82
1997     8.03
1998     7.34
1999     7.09
2000     6.99
2001     6.99
2002     7.54
2003     7.90
2004     8.06
2005     8.06
2006     7.01
2007     6.69
2008     6.73
2009     6.27
2010     7.22
2011     8.49
2012     8.48
2013     9.69
2014    10.63
2015    10.07
2016     9.55
2017     8.98
2018     9.05
Name: MESTRADO, dtype: float64

In [96]:
capes_bolsas['IES'].unique()

array(['UFAC', 'UNEAL', 'UFAL', 'IFAL', 'UNIT/ALAGOAS', 'FIOCRUZ-CPQLMD',
       'INPA', 'UEA', 'UFAM', 'UNINILTON', 'IFAM', 'UNIFAP', 'UFOB',
       'UFRB', 'UEFS', 'UNIFG', 'UESC', 'UFESBA', 'UFBA', 'UNEB', 'EBMSP',
       'UCSAL', 'UNIFACS', 'IFBAIANO-CATU', 'IFBA', 'IFBAIANO',
       'FIOCRUZ-CPQGM', 'UESB', 'URCA', 'IFCE', 'UECE', 'UFC', 'UNIFOR',
       'UFCA', 'UNILAB', 'UVA-CE', 'UNINTA', 'UNB', 'UCB', 'UNICEUB',
       'IFB', 'UNIEURO', 'UVV', 'UFES', 'FUCAPE', 'IFES', 'EMESCAM',
       'UEG', 'UNIEVANGELICA', 'IFGOIANO', 'UFG', 'PUC-GOIAS', 'IFG',
       'FESURV', 'IFMA - MC', 'UEMA', 'UFMA', 'UNIFAL', 'UNIFENAS',
       'CEFET/MG', 'FJP', 'UEMG', 'UFMG', 'PUC/MG', 'FAJE', 'IEPSC',
       'IFMG', 'FIOCRUZ-CPQRR', 'CDTN', 'UNIHORIZONTES', 'UFVJM',
       'UNIVALE', 'UNIFEI', 'UFJF', 'IFSUDESTEMG', 'UFLA', 'UNIMONTES',
       'IFNMG', 'FDMC', 'UFOP', 'FDSM', 'UNIVAS', 'IFSULDEMINAS',
       'INATEL', 'UFSJ', 'UNINCOR', 'UFTM', 'UNIUBE', 'IFTM', 'UFU',
       'UFV', 'UFMS', 'UCD

In [97]:
# vimos que temos universidades que estão com - e outras com /, transformar tudo como /
capes_bolsas['IES'] = capes_bolsas['IES'].str.replace('-', '/').str.replace('/', '/').str.replace(' ','/')


In [98]:
## para mantermos somente o IES puro, sem o nome do campos, o campos iremos utilizar pelo municipio, pois existem momentos
## no banco de dados que alguns tem campos e outros não, o que está diferenciando a mesmo IES para isso iremos padronizar e olhar os campos pelo MUN
capes_bolsas['IES'] = capes_bolsas['IES'].str.split('/').str[0]

In [99]:
#capes_bolsas.to_csv('capes_bolsas_py.csv', index=False)

ANO                   int32
UF                   object
MUN                  object
REGIAO               object
COD_PROG             object
PROG_FORMENTO        object
IES                  object
STATUS_JURIDICO      object
PROGRAMA             object
AREA_AVALIACAO       object
AREA_CONHECIMENTO    object
GRANDE_AREA          object
DOUTORADO             int64
MESTRADO              int64
dtype: object


,ANO,UF,MUN,REGIAO,COD_PROG,PROG_FORMENTO,IES,STATUS_JURIDICO,PROGRAMA,AREA_AVALIACAO,AREA_CONHECIMENTO,GRANDE_AREA,DOUTORADO,MESTRADO
0,2018,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,FEDERAL,NaN,INDEFINIDA,GRANDE AREA NAO INFORMADA,GRANDE AREA NAO INFORMADA,1,38
4,2018,AC,RIO BRANCO,NORTE,11001011001P8,DS,UFAC,FEDERAL,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,INDEFINIDA,ECOLOGIA,CIENCIAS BIOLOGICAS,0,8
6,2018,AC,RIO BRANCO,NORTE,11001011003P0,DS,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,INDEFINIDA,LETRAS,"LINGUISTICA, LETRAS E ARTES",0,17
8,2018,AC,RIO BRANCO,NORTE,11001011004P7,DS,UFAC,FEDERAL,PRODUCAO VEGETAL,INDEFINIDA,AGRONOMIA,CIENCIAS AGRARIAS,18,9
9,2018,AC,RIO BRANCO,NORTE,11001011004P7,FAPAC,UFAC,FEDERAL,PRODUCAO VEGETAL,INDEFINIDA,AGRONOMIA,CIENCIAS AGRARIAS,7,0
